In [1]:
import numpy as np
from fractions import Fraction
import cdd
from polytope import polytope
from polytope.polytope import qhull
from scipy.spatial import ConvexHull

Let's construct a reflexive 3D polytope

In [2]:
V = np.array([[-1, -2,  2], [ 1,  2, -2], [-3, -2,  0], [ 1,  2, -3], [ 0, -1,  2]])

In [7]:
#code based on https://github.com/stephane-caron/pypoman/blob/master/pypoman/duality.py
def compute_distances(vertices: np.ndarray) -> np.ndarray:
    V = np.vstack(vertices)
    t = np.ones((V.shape[0], 1))  # first column is 1 for vertices
    tV = np.hstack([t, V])
    mat = cdd.Matrix(tV, number_type="fraction")
    mat.rep_type = cdd.RepType.GENERATOR
    P = cdd.Polyhedron(mat)
    bA = np.array(P.get_inequalities())
    if bA.shape == (0,):  # bA == []
        return bA
    # the polyhedron is given by b + A x >= 0 where bA = [b|A]
    b = bA[:, 0]
    A = -np.array(bA[:, 1:])

    for i, row in enumerate(A):
        denominators = []
        has_fraction = False
        for j in row:
            if type(j) == Fraction:
                has_fraction = True
                denominators.append(j.denominator)    
        if has_fraction:
            b[i] *= np.lcm.reduce(denominators)
    return b

def calc_fitness(vertices):
    ip_count = len(polytope.enumerate_integral_points(qhull(vertices)))
    distances = compute_distances(vertices)
    result = 0
    if ip_count > 1:
        result -= 1
    for d in distances:
        result -= abs(d-1)
    return result

integral_points = polytope.enumerate_integral_points(qhull(V))
integral_points = integral_points.transpose()
integral_points = integral_points.astype(int)

print("integral_points: {0}".format(integral_points.tolist()))
print("distance vector: {0}".format(compute_distances(V)))
print("fitness: {0}".format(calc_fitness(V)))

integral_points: [[0, 0, 0]]
distance vector: [1 2 1 1 1 1]
fitness: -2
